In [1]:
import spacy
import numpy as np
import pandas as pd
from stopwords import ENGLISH_STOP_WORDS
import warnings
from __future__ import unicode_literals
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
en_nlp = spacy.load('en')

In [50]:
def spacy_get_vec(sentence):
    vec = np.zeros(768)
    doc = en_nlp((sentence))

    allowed_words = set(['not', 'what', 'why', 'how', 'where'])
    for i,word in enumerate(doc):
        if i < len(doc) -1:
            tempvec = np.append(word.vector, doc[i + 1].vector)
            vec += tempvec
        else:
            tempvec = np.append(word.vector, np.zeros(384))
            vec += tempvec
    return vec

In [ ]:
senti=pd.read_csv('/SentiWordNet_3.0.0_20130122.csv',sep='\\t')

In [75]:
senti['equil']=senti['PosScore']-senti['NegScore']
score=[]
examples=[]
del(i)
finalex=pd.DataFrame()
senti=senti[0:500]
for i in range(500):
    if senti.loc[i]['equil']>0:
        result='pos'
        score.append('pos')
    if senti.loc[i]['equil']<0:
        result='neg'
        score.append('neg')
    if senti.loc[i]['equil']==0:
        result='neu'
        score.append('neu')
    for ex in senti.loc[i]['Gloss'].split(';')[1:2]:
        examples.append(ex)
    if (examples!=[]):
        examples.append(result)
        df = pd.DataFrame(examples)
        finalex= finalex.append(df.transpose(),ignore_index=True)
        examples=[]
senti['res']=score
senti[:5]

,# POS,ID,PosScore,NegScore,SynsetTerms,Gloss,equil,res
0,"""a",1740.0,0.125,0.00,able#1,(usually followed by `to') having the necessar...,0.125,pos
1,"""a",2098.0,0.000,0.75,unable#1,(usually followed by `to') not having the nece...,-0.750,neg
2,"""a",2312.0,0.000,0.00,dorsal#2 abaxial#1,facing away from the axis of an organ or organ...,0.000,neu
3,"""a",2527.0,0.000,0.00,ventral#2 adaxial#1,nearest to or facing toward the axis of an org...,0.000,neu
4,a,2730.0,0.000,0.00,acroscopic#1,facing or on the side toward the apex,0.000,neu


In [76]:
vecs = []
intents = []
idfs = []
for i in range(500):
    tokens = senti.loc[i]['SynsetTerms'].split('#')
    sentence = tokens[0]
    intent = senti.loc[i]['res']
    '''if intent[-1] == '\n':
        intent = intent[:-1]'''
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)
    #idfs.append(get_idf(sentence))

df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(768)])
df['intents'] = intents


X = df.iloc[:, :-1].values
y = df.iloc[:,-1:].values.ravel()

In [77]:
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.20)

In [78]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', degree=2, class_weight={'pos':1.1,'neg':1.1,'neu':0.8}, probability=True)
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200,
  class_weight={u'neg': 1.1, u'neu': 0.8, u'pos': 1.1}, coef0=0.0,
  decision_function_shape=None, degree=2, gamma='auto', kernel=u'rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [79]:
print(svc.score(X_train, y_train))
print(svc.score(X_val, y_val))

0.545
0.52


In [80]:
from sklearn.ensemble import GradientBoostingClassifier
gradboost = GradientBoostingClassifier(n_estimators=500, max_depth=25, max_features='sqrt')

In [81]:
gradboost.fit(X_train, y_train)
print(gradboost.classes_)
print(gradboost.score(X_train, y_train))
print(gradboost.score(X_val, y_val))

[u'neg' u'neu' u'pos']
0.98
0.51


In [82]:
gradboost.predict_proba(spacy_get_vec('Live is not awesome'))

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.67031213,  0.04757537,  0.2821125 ]])

In [83]:
gradboost.classes_

array([u'neg', u'neu', u'pos'], dtype=object)

In [86]:
svc.predict(spacy_get_vec('I am feeling vey sad'))

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([u'neg'], dtype=object)

In [87]:
from sklearn.externals import joblib
joblib.dump(svc, 'sentiment.pkl')

[u'sentiment.pkl']